In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os

# Read the HTML file
with open('Flipkart.html', 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

products = []
prices = []

# Each product card is inside div[data-id]
divs = soup.find_all('div', {'data-id': True})

for div in divs:
    # --- Product Name ---
    title_tag = div.select_one("wjcEIp")  # product link
    if not title_tag:
        img_tag = div.select_one("div._4WELSP img")
        product_name = img_tag['alt'].strip() if img_tag and img_tag.has_attr("alt") else ""
    else:
        product_name = title_tag.get_text(strip=True)

    # --- Product Price ---
    price_tag = div.select_one("div.Nx9bqj")
    product_price = price_tag.get_text(strip=True) if price_tag else ""

    # ✅ Filter: Only keep items where "ROSIER" is in product name
    if "ROSIER" in product_name.upper():
        products.append(product_name)
        prices.append(product_price)

# DataFrame बनाओ
df = pd.DataFrame({
    'Product_Name': products,
    'Product_Price': prices
})

df.to_excel('flipkart_products.xlsx', index=False)

print("Scraping complete. Saved flipkart_products.xlsx")


Scraping complete. Saved flipkart_products.xlsx


In [3]:
# Save to Excel
excel_file = 'Flipkart_products.xlsx'
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.title = "Rosier Products"

In [4]:
# Email setup
email_address = "Priyanshkhandelwal946215@gmail.com"
password = "iksg bgeq ylly gyyz"  # Replace with your app-specific password

In [5]:
# Create email
msg = MIMEMultipart()
msg['From'] = email_address
msg['To'] = email_address
msg['Subject'] = 'Flipkart Product Data'

In [6]:
# Attach Excel file
with open(excel_file, 'rb') as f:
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(f.read())
part.add_header('Content-Disposition', f'attachment; filename={excel_file}')
encoders.encode_base64(part)
msg.attach(part)

In [7]:
# Send email
try:
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(email_address, password)
    server.sendmail(email_address, email_address, msg.as_string())
    server.quit()
    print("Email sent successfully")
except Exception as e:
    print(f"Error sending email: {e}")

Email sent successfully
